## 前置處理

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import KFold


# 超參數
num_epochs = 5
batch_size = 64
learning_rate = 0.001
num_folds = 5

# MNIST數據預處理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)




## 建立 ANN

In [3]:

# 定義簡單的ANN模型
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

## Kfold with ANN

In [ ]:
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold = 1

for train_indices, val_indices in kf.split(mnist_dataset):
    print(f"Fold {fold}")
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset=mnist_dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset=mnist_dataset, batch_size=batch_size, sampler=val_sampler)

    model = ANN()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Accuracy: {accuracy:.4f}')

    fold += 1

## ANN With Dropout

In [4]:
# 定義具有Dropout的ANN模型
class ANNWithDropout(nn.Module):
    def __init__(self):
        super(ANNWithDropout, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # 添加Dropout，丟棄50%的神經元
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)  # 在這一層後應用Dropout
        x = self.fc2(x)
        return x

model = ANNWithDropout()


## KFold with ANNWithDropout

In [ ]:
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold = 1

for train_indices, val_indices in kf.split(mnist_dataset):
    print(f"Fold {fold}")
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset=mnist_dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset=mnist_dataset, batch_size=batch_size, sampler=val_sampler)

    model = ANNWithDropout()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Accuracy: {accuracy:.4f}')

    fold += 1